# DESaster Vers -  0.1

In [11]:
import sys, random
desaster_path = "/Users/geomando/Dropbox/github/SeaGrantSimulation"
sys.path.append(desaster_path)
import simpy
import pandas as pd
import numpy as np
import desaster
from desaster import request, entities, rebuild, config, io


##### instantiate a simpy environment

In [12]:
sim = simpy.Environment()

In [13]:
def simulation(sim, res, entity):
    
    yield sim.process(request.inspection(entity, 
                                       sim, 
                                       res))
    #print ("Inspection Started for {0} at {1}".format(entity.name, sim.now))
    
    yield sim.process(request.file_insurance_claim(entity,
                                         sim,
                                         res))
    #print ("Insurance Started for {0} at {1}".format(entity.name, sim.now))
    
    yield (sim.process(request.fema_assistance(entity,
                                            sim,
                                            res)) &
           
          sim.process(request.loan(entity, 
                                   sim, 
                                   res)) &
                                            #THESE ARE ALL IN THE SAME YIELD STATEMENT
          sim.process(request.permit(entity, 
                                     sim, 
                                     res)))
    #print ("PreBuilding processes Started for {0} at {1}".format(entity.name, sim.now))
    
    
    yield sim.process(rebuild.rebuild_house(entity,
                                sim,
                                res.contractors))
    #print ("Rebuild Started for {0} at {1}".format(entity.name, sim.now))
    
    yield sim.process(request.engineering_assessment(entity,
                                                    sim,
                                                    res))
    #print ("Engineering assessment Started for {0} at {1}".format(entity.name, sim.now))
    

In [14]:
human_cap_data = {"inspectors": 100,
        "insurance adjusters": 10,
        "fema processors": 30,
        "permit processors": 3,
        "contractors": 25,
        "loan processors": 2,
        "engineers": 15,
        "fema aid": 100000000}


In [15]:
res = desaster.Capitals(sim, human_cap_data)

In [16]:
d = io.view_config()

adjuster_mean = 90.0
adjuster_std = 0.0
adjuster_time = 90.0
engineering_assessment_time = 14.0
engineering_mean = 14.0
engineering_std = 0.0
fema_process_mean = 60.0
fema_process_std = 0.0
fema_process_time = 60.0
inspection_mean = 14.0
inspection_std = 0
inspection_time = 14.0
loan_process_mean = 90.0
loan_process_std = 0.0
loan_process_time = 90.0
permit_process_mean = 30.0
permit_process_std = 0.0
permit_process_time = 30.0


In [21]:
input_path = "../inputs/Final_Maura_Inputs.csv"
households_df = pd.read_csv(input_path)
households_df.head(3)

,Address,Lat,Long,NEAR_FID,NEAR_DIST,FID_1,ID_Number,Tract,Name,YearBuilt,...,None,Slight,Moderate,Extensive,Complete,At_Least_S,At_Least_M,At_Least_E,TotalRep,Damaged
0,1 ULLAKKO RD,46.350970,-123.812630,2993,0.000727,2993,US002994,5.304995e+10,"HILL, DOROTHY M",1975,...,0.004,0.08,0.578,0.327,0.01,0.996,0.916,0.337,68700.0,1
1,10 KOVEN ROAD,46.367306,-123.816740,203,0.000550,203,US000204,5.304995e+10,"LEINWEBER, MICHAEL D & THERESA",1975,...,0.004,0.08,0.578,0.327,0.01,0.996,0.916,0.337,119100.0,1
2,10 RYE LANE,46.587673,-123.619364,2934,0.003318,2934,US002935,5.304995e+10,"MARTIN, PATRICIA L",1969,...,0.004,0.08,0.578,0.327,0.01,0.996,0.916,0.337,49950.0,1


In [22]:
household = {}
for i in households_df.index:
    household[i] = entities.Household(households_df.loc[i])

In [23]:
for i in range(len(household)):
    #print (household[i])
    sim.process(simulation(sim, res, household[i]))

In [24]:
sim.run()

AttributeError: 'Household' object has no attribute 'rebuild_time'

In [ ]:
household[0].__getattribute__("claim_get")

In [ ]:
import pprint
length = (len(household))
#for i in range(length):
#    print (household[i].household.Damaged)
count = 0
for j in range(length):
    #print (j)
    for i in dir(household[j]):
        if "get" in i and household[j].assessment_get < 500:
            print (household[j].household.Name)
            print(household[j].__getattribute__(i), " ", i)
            print("-------------------------")
            count += 1
            print (count)

In [ ]:
len(res.housing_stock.items)

In [ ]:
vars((household[0]))
